In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.nn.functional as F

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer,
    StandardScaler,
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import copy

In [2]:
import os


# Common  columns : Crop,Year


def preprocess_data(df):

    df.drop(
        axis=1,
        columns=[
            "districtcode",
            "statecode",
            "year",
            "crop",
            "districtname",
        ],
        inplace=True,
    )

    region_map = {
        "Chandigarh": "North", "Delhi": "North", "Haryana": "North", "Himachal Pradesh": "North",
        "Jammu & Kashmir": "North", "Punjab": "North", "Rajasthan": "North",
        "Arunachal Pradesh": "North-East", "Assam": "North-East", "Manipur": "North-East",
        "Meghalaya": "North-East", "Mizoram": "North-East", "Nagaland": "North-East", "Tripura": "North-East",
        "Andaman & Nicobar Islands": "East", "Bihar": "East", "Jharkhand": "East",
        "Odisha": "East", "Sikkim": "East", "West Bengal": "East",
        "Chhattisgarh": "Central", "Madhya Pradesh": "Central", "Uttar Pradesh": "Central", "Uttarakhand": "Central",
        "Dadra & Nagar Haveli": "West", "Daman & Diu": "West", "Goa": "West",
        "Gujarat": "West", "Maharashtra": "West",
        "Andhra Pradesh": "South", "Karnataka": "South", "Kerala": "South",
        "Lakshadweep": "South", "Puducherry": "South", "Tamil Nadu": "South"
    }

    # Drop rows with unmatched states
    
    df["region"] = df["statename"].map(region_map)
    
    df = df.dropna(subset=["region"])
    
    df.drop(axis =1 ,columns=['statename'],inplace=True)
    print(f"lenght of dataframe is {len(df)}")

    categorical_features = ["region"]

    cobb_douglas_features = [
        "area1000hectares",
        "irrigatedarea1000hectares",
        "unirrigatedarea1000hecatres",
        "nitrogenconsumptiontonnes",
        "phosphateconsumptiontonnes",
        "potashconsumptiontonnes",
        "production1000tonnes",
    ]

    env_features = [
        "total_rainfall",
        "average_rainfall",
        "salinity_alkalinity_percent",
    ]

    df["unirrigatedarea1000hecatres"] = (
        df["area1000hectares"] - df["irrigatedarea1000hectares"] + 1
    )

    df["irrigatedarea1000hectares"] = df["irrigatedarea1000hectares"] + 1

    df["salinity_alkalinity_percent"] = df["salinity_alkalinity_percent"] + 1

    log_transform = Pipeline(
        steps=[
            ("log", FunctionTransformer(np.log, validate=True)),
        ]
    )

    categorical_transformer = Pipeline(
        steps=[("onehot", OneHotEncoder( handle_unknown="ignore"))]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", log_transform, cobb_douglas_features),
            ("cat", categorical_transformer, categorical_features),
            ("passthrough", "passthrough", env_features)
        ],sparse_threshold=0.0,remainder="drop"
    )
    x_transform = preprocessor.fit_transform(df)
    cat_column_names = preprocessor.named_transformers_["cat"]["onehot"].get_feature_names_out(categorical_features)
    X_df = pd.DataFrame(x_transform, columns=cobb_douglas_features + env_features + list(cat_column_names))

    output_file = "output.txt"
    with open(output_file, "w") as f:
        for i in X_df.iloc[0].items():

            f.write(str(i))
            f.write("\n")

    X = X_df.drop(axis=1, columns=["production1000tonnes"])
    # print(X.describe())
    Y = X_df["production1000tonnes"]
    return X, Y


path_to_dataset = "Dataset/Task2Summary.csv"
df = pd.read_csv(path_to_dataset)


X_copy, Y_Copy = preprocess_data(df)
X, Y = torch.tensor(X_copy.to_numpy()), torch.tensor(Y_Copy.to_numpy())

lenght of dataframe is 246


/tmp/ipykernel_214768/1993380002.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(axis =1 ,columns=['statename'],inplace=True)
/tmp/ipykernel_214768/1993380002.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["unirrigatedarea1000hecatres"] = (
/tmp/ipykernel_214768/1993380002.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [3]:
class OLS(nn.Module):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.num_features = None
        self.weights = None
        self.bias = None

    def forward(self, X):
        if self.num_features == None:
            raise LookupError("Train the OLS First :)")

        # X = (X - X.mean(dim=0, keepdim=True)) / X.std(dim=0, keepdim=True)
        return torch.matmul(X, self.weights) + self.bias

    def calculate_statistics(self, y_prime: torch.tensor, y: torch.tensor):
        SST = ((y - y.mean()) ** 2).sum()

        SSR = ((y_prime - y.mean()) ** 2).sum()

        SSE = ((y_prime - y) ** 2).sum()

        EXPECTEDVALUE_U = (y - y_prime).sum()

        R2 = 1 - (SSE / SST)

        n = y.shape[0]
        p = self.num_features  
        adj_R2 = 1 - ((1 - R2) * (n - 1)) / (n - p - 1)

        print(
            f"Sum of squared Terms {SST.item():.4f}\nSum of squared errors {SSE.item():.4f} \nRegression sum of squares {SSR.item():.4f} \nExpected value of U {EXPECTEDVALUE_U.item():.4f} \nR2 {R2} \nadjustedR2 {adj_R2}"
        )
        return SSR, SSE, SST

    def fit(self, dataset):

        # Only accepts full batch
        X,Y = dataset
        Y = Y.unsqueeze(1)
        n, p = X.shape
        X_aug = torch.cat([torch.ones(n,1), X], dim=1)   # (n, p+1)
        # closed‑form
        w_full = torch.linalg.pinv(X_aug) @ Y            # (p+1,1)

        self.bias    = nn.Parameter(w_full[0:1, :])       # (1,1)
        self.weights = nn.Parameter(w_full[1:, :])       # (p,1)
        self.num_features = p
        print(f"Trained OLS for {self.num_features} features")
        print(f"Beta matrix is {self.weights} (without beta not) ,Beta not = { self.bias }")
        print('\n')
        self.calculate_statistics(self(X), Y)

In [4]:
model = OLS()
model.fit((X, Y))

Trained OLS for 15 features
Beta matrix is Parameter containing:
tensor([[ 9.7237e-01],
        [ 2.2199e-02],
        [-1.8103e-01],
        [-1.7440e-02],
        [ 8.8580e-02],
        [-1.1508e-02],
        [-9.1308e-03],
        [ 1.4772e-01],
        [ 2.8890e-01],
        [-2.3385e-01],
        [-2.6597e-01],
        [-5.7998e-01],
        [-2.0176e-04],
        [ 7.9323e-03],
        [ 2.4529e-03]], dtype=torch.float64, requires_grad=True) (without beta not) ,Beta not = Parameter containing:
tensor([[-0.6523]], dtype=torch.float64, requires_grad=True)


Sum of squared Terms 621.9362
Sum of squared errors 65.2809 
Regression sum of squares 556.6553 
Expected value of U 0.0000 
R2 0.8950360178866549 
adjustedR2 0.8881905407923063


Useful for later(q8)

In [5]:
SSR_r = torch.sum((Y - model(X)) ** 2).item()  # base model
k_r = model.weights.shape[0]# base model parameters
n = X.shape[0]
df_r = n - k_r

In [6]:
# This is for EDA

FILE_NAME = "Task2Summary.csv"

df = pd.read_csv(FILE_NAME)


print("Dataset Info:")
print(df.info())

print("Summary Statistics:")
print(df.describe())

print("Missing Values:")
print(df.isnull().sum())

print("Duplicate Rows:", df.duplicated().sum())

if "target" in df.columns:
    print("Target Distribution:")
    print(df["target"].value_counts())
    sns.countplot(data=df, x="target")
    plt.title("Target Class Distribution")
    plt.show()

plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm")
plt.title("Correlation Matrix")
plt.show()

df.hist(figsize=(12, 8), bins=30)
plt.suptitle("Histogram of Features", fontsize=16)
plt.tight_layout()
plt.show()

numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
for col in numerical_cols:
    plt.figure(figsize=(6, 3))
    sns.boxplot(data=df, x=col)
    plt.title(f"Boxplot: {col}")
    plt.tight_layout()
    plt.show()


# cat_cols = df.select_dtypes(include="object").columns.tolist()
# for col in cat_cols:
#     print(f"Value Counts for {col}:")
#     print(df[col].value_counts())
#     sns.countplot(data=df, x=col)
#     plt.title(f"Distribution of {col}")
#     plt.xticks(rotation=45)
#     plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'Task2Summary.csv'

✅ Question 6: Diminishing Marginal Product (Quadratic Model)
From the quadratic model regression:

Squared terms like area1000hectares_sq, irrigatedarea1000hectares_sq, nitrogenconsumptiontonnes_sq, etc., mostly have negative coefficients.
These negative signs indicate diminishing returns—as input increases, additional output increases at a decreasing rate.
Many of these squared terms are statistically significant (p-value < 0.05), confirming that:
✅ We reject the null hypothesis for those inputs:

"Marginal productivity diminishes as input levels rise beyond a threshold."
✅ Question 7: Complementarity of Irrigation and Fertilizer
From the interaction model:

The coefficient of the interaction term irrigation_x_nitrogen is:
Positive, suggesting that increased irrigation amplifies the effect of nitrogen application.
Statistically significant (p-value < 0.05), providing evidence for input complementarity.
✅ We reject the null hypothesis:

"There is a positive interaction between irrigation and nitrogen — they are complementary inputs."

/Users/dhruvyadav/Downloads/Task2Summary.csv

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("Dataset/Task2Summary.csv")

# Drop irrelevant columns
df.drop(
    axis=1,
    columns=["districtcode", "statename", "statecode", "year", "crop", "districtname"],
    inplace=True,
)

# Base input features
base_features = [
    "area1000hectares", "production1000tonnes", "irrigatedarea1000hectares",
    "nitrogenconsumptiontonnes", "phosphateconsumptiontonnes", "potashconsumptiontonnes",
    "total_rainfall", "average_rainfall", "salinity_alkalinity_percent"
]

# Add unirrigated area
df["unirrigatedarea1000hecatres"] = df["area1000hectares"] - df["irrigatedarea1000hectares"]

# Add squared terms for quadratic model
for feature in base_features:
    if feature != "production1000tonnes":
        df[f"{feature}_squared"] = df[feature] ** 2

# Define inputs and output
X = df.drop(columns=["production1000tonnes"])
Y = df["production1000tonnes"]


This is the model to test for diminishing marginal product of inputs by checking if squared coefficients are negative and significant.
For each input, if its squared term has:
Negative coefficient ✅
p-value < 0.05 ✅
→ Then it shows diminishing marginal returns for that input.
Question 6

In [ ]:
import statsmodels.api as sm

# Add constant/intercept
X_quad_const = sm.add_constant(X)

# Fit the quadratic OLS model
model_quad = sm.OLS(Y, X_quad_const).fit()

# Display results
print(model_quad.summary())


                             OLS Regression Results                             
Dep. Variable:     production1000tonnes   R-squared:                       0.878
Model:                              OLS   Adj. R-squared:                  0.870
Method:                   Least Squares   F-statistic:                     106.8
Date:                  Tue, 22 Apr 2025   Prob (F-statistic):           8.00e-99
Time:                          13:33:57   Log-Likelihood:                -913.49
No. Observations:                   255   AIC:                             1861.
Df Residuals:                       238   BIC:                             1921.
Df Model:                            16                                         
Covariance Type:              nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------

If irrigation_x_nitrogen coefficient is:
Positive ✅
Statistically significant (p < 0.05) ✅
→ Then there's positive complementarity between irrigation and nitrogen.

In [ ]:
# Add interaction term: irrigation × nitrogen
X["irrigation_x_nitrogen"] = X["irrigatedarea1000hectares"] * X["nitrogenconsumptiontonnes"]

# Fit OLS model with interaction term
X_interact_const = sm.add_constant(X)
model_interact = sm.OLS(Y, X_interact_const).fit()

# Display results
print(model_interact.summary())


                             OLS Regression Results                             
Dep. Variable:     production1000tonnes   R-squared:                       0.879
Model:                              OLS   Adj. R-squared:                  0.871
Method:                   Least Squares   F-statistic:                     101.7
Date:                  Tue, 22 Apr 2025   Prob (F-statistic):           1.63e-98
Time:                          13:34:12   Log-Likelihood:                -911.71
No. Observations:                   255   AIC:                             1859.
Df Residuals:                       237   BIC:                             1923.
Df Model:                            17                                         
Covariance Type:              nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------

If region dummy coefficients are significant → there are statistical differences across regions.
Question 8


Q8

Model-A (Enhanced)

In [ ]:
import os


# Common  columns : Crop,Year
def preprocess_data(df):
    df = df.copy()  # Always work on a fresh copy to avoid warnings

    # Adjust irrigated area to handle log(0)
    df["irrigatedarea1000hectares"] += 1
    df["unirrigatedarea1000hecatres"] = df["area1000hectares"] - df["irrigatedarea1000hectares"]

    # Add region column
    region_map = {
        "Chandigarh": "North", "Delhi": "North", "Haryana": "North", "Himachal Pradesh": "North",
        "Jammu & Kashmir": "North", "Punjab": "North", "Rajasthan": "North",
        "Arunachal Pradesh": "North-East", "Assam": "North-East", "Manipur": "North-East",
        "Meghalaya": "North-East", "Mizoram": "North-East", "Nagaland": "North-East", "Tripura": "North-East",
        "Andaman & Nicobar Islands": "East", "Bihar": "East", "Jharkhand": "East",
        "Odisha": "East", "Sikkim": "East", "West Bengal": "East",
        "Chhattisgarh": "Central", "Madhya Pradesh": "Central", "Uttar Pradesh": "Central", "Uttarakhand": "Central",
        "Dadra & Nagar Haveli": "West", "Daman & Diu": "West", "Goa": "West",
        "Gujarat": "West", "Maharashtra": "West",
        "Andhra Pradesh": "South", "Karnataka": "South", "Kerala": "South",
        "Lakshadweep": "South", "Puducherry": "South", "Tamil Nadu": "South"
    }
    df["region"] = df["statename"].map(region_map)

    # Drop rows with unmatched states
    df = df.dropna(subset=["region"])
    df = df.drop(columns=["statename", "districtcode", "statecode", "districtname", "year", "crop"])

    # Define columns
    log_features = [
        "area1000hectares",
        "irrigatedarea1000hectares",
        "unirrigatedarea1000hecatres",
        "nitrogenconsumptiontonnes",
        "phosphateconsumptiontonnes",
        "potashconsumptiontonnes"
    ]
    environmental_features = ["total_rainfall", "average_rainfall", "salinity_alkalinity_percent"]

    # Replace 0 or negative values with small constant before log
    for col in log_features:
        df[col] = df[col].clip(lower=1e-6)
        df[f"log_{col}"] = np.log(df[col])

    # Log transform output
    df["production1000tonnes"] = df["production1000tonnes"].clip(lower=1e-6)
    df["log_production"] = np.log(df["production1000tonnes"])

    # One-hot encode regions (drop one category for baseline)
    region_dummies = pd.get_dummies(df["region"], drop_first=True)
    df = df.join(region_dummies)

    # Add interaction terms
    for var in log_features:
        for region in region_dummies.columns:
            df[f"log_{var}_x_{region}"] = df[f"log_{var}"] * df[region]

    # Final feature set
    feature_cols = (
        [f"log_{col}" for col in log_features] +
        environmental_features +
        region_dummies.columns.tolist() +
        [f"log_{var}_x_{region}" for var in log_features for region in region_dummies.columns]
    )

    # Make sure all columns are float32
    X = df[feature_cols].astype(np.float32)
    Y = df["log_production"].astype(np.float32)

    return X, Y

path_to_dataset = "Task2Summary.csv"
df = pd.read_csv(path_to_dataset)

X_df1, Y_df = preprocess_data(df)

# Convert to torch tensors
X_d, Y = torch.tensor(X_df1.to_numpy(), dtype=torch.float32), torch.tensor(Y_df.to_numpy(), dtype=torch.float32)

model_a_enhanced = OLS() 
model_a_enhanced.fit((X_d, Y))


torch.Size([])
Trained OLS for 44 features
Sum of squared Terms 621.9362
Sum of squared errors 294.0233 
Regression sum of squares 831.9655 
Expected value of U 36.3207 
R2 0.527245283126831 
adjustedR2 0.4237567186355591


Model-B(Enhanced)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Load the dataset
df = pd.read_csv("Task2Summary.csv")
df = df.copy()

# Add unirrigated area
df["unirrigatedarea1000hecatres"] = df["area1000hectares"] - df["irrigatedarea1000hectares"]

# Add region classification
region_map = {
    "Chandigarh": "North", "Delhi": "North", "Haryana": "North", "Himachal Pradesh": "North",
    "Jammu & Kashmir": "North", "Punjab": "North", "Rajasthan": "North",
    "Arunachal Pradesh": "North-East", "Assam": "North-East", "Manipur": "North-East",
    "Meghalaya": "North-East", "Mizoram": "North-East", "Nagaland": "North-East", "Tripura": "North-East",
    "Andaman & Nicobar Islands": "East", "Bihar": "East", "Jharkhand": "East",
    "Odisha": "East", "Sikkim": "East", "West Bengal": "East",
    "Chhattisgarh": "Central", "Madhya Pradesh": "Central", "Uttar Pradesh": "Central", "Uttarakhand": "Central",
    "Dadra & Nagar Haveli": "West", "Daman & Diu": "West", "Goa": "West",
    "Gujarat": "West", "Maharashtra": "West",
    "Andhra Pradesh": "South", "Karnataka": "South", "Kerala": "South",
    "Lakshadweep": "South", "Puducherry": "South", "Tamil Nadu": "South"
}
df["region"] = df["statename"].map(region_map)
df = df.dropna(subset=["region"])  # Drop rows where region is not assigned

# Base features
features = [
    "area1000hectares", "irrigatedarea1000hectares", "nitrogenconsumptiontonnes",
    "phosphateconsumptiontonnes", "potashconsumptiontonnes",
    "total_rainfall", "average_rainfall", "salinity_alkalinity_percent",
    "unirrigatedarea1000hecatres"
]

# Add squared terms
for f in features:
    df[f"{f}_squared"] = df[f] ** 2

# Add region dummies (drop one region for baseline)
region_dummies = pd.get_dummies(df["region"], drop_first=True)
df = df.join(region_dummies)

# Create interaction terms between region and each feature and squared term
for f in features + [f"{f}_squared" for f in features]:
    for region in region_dummies.columns:
        df[f"{f}_x_{region}"] = df[f] * df[region]

# Define target variable
Y = df["production1000tonnes"]
'''
# Model B (base quadratic model without region interaction)
X_base = df[
    features + 
    [f"{f}_squared" for f in features]
]
X_base = sm.add_constant(X_base)
model_base = sm.OLS(Y, X_base).fit()
'''
# Enhanced Model B (with region interaction terms)
interaction_cols = [col for col in df.columns if "_x_" in col]
X_enhanced = df[
    features + 
    [f"{f}_squared" for f in features] +
    interaction_cols
]
X_enhanced = sm.add_constant(X_enhanced)
model_enhanced = sm.OLS(Y, X_enhanced).fit()

# Quadratic Model (refit and print summary of base model)
X_quad = df[features + [f"{f}_squared" for f in features]]
X_quad = sm.add_constant(X_quad)

# Ensure index alignment
X_quad, Y = X_quad.align(Y, join='inner', axis=0)

model_quad = sm.OLS(Y, X_quad).fit()
print(model_quad.summary())


                             OLS Regression Results                             
Dep. Variable:     production1000tonnes   R-squared:                       0.877
Model:                              OLS   Adj. R-squared:                  0.867
Method:                   Least Squares   F-statistic:                     95.22
Date:                  Tue, 22 Apr 2025   Prob (F-statistic):           2.55e-93
Time:                          13:02:56   Log-Likelihood:                -885.42
No. Observations:                   246   AIC:                             1807.
Df Residuals:                       228   BIC:                             1870.
Df Model:                            17                                         
Covariance Type:              nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------

In [ ]:
# F-test to compare Part B Models
f_test_result = model_enhanced.compare_f_test(model_quad)
print("F-test result (F-statistic, p-value, degrees of freedom):", f_test_result)

F-test result (F-statistic, p-value, degrees of freedom): (np.float64(2.1984074035452514), np.float64(3.059410660341792e-05), np.float64(66.0))


In [ ]:
import scipy.stats as stats
# Make sure Y is a PyTorch tensor
Y_tensor = torch.tensor(Y.values, dtype=torch.float32).view(-1, 1)

# SSR for enhanced model
SSR_e = torch.sum((Y_tensor - model_a_enhanced(X_d)) ** 2).item()
k_e = model_a_enhanced.weights.shape[0]  # enhanced model parameters (including bias)
df_e = n - k_e  # degrees of freedom for the enhanced model
# Calculate the F-statistic
F_stat = ((SSR_r - SSR_e) / (k_e - k_r)) / (SSR_e / df_e)
p_value = 1 - stats.f.cdf(F_stat, dfn=(k_e - k_r), dfd=(n - k_e))
#p_value = 1 - stats.f.cdf(F_stat, k_e - k_r, df_e)

print(f"F-statistic: {F_stat:.4f}")
print(f"P-value: {p_value:.4f}")

F-statistic: -6.0286
P-value: 1.0000
